In [10]:
import gym
import ptan
import ptan.ignite as ptan_ignite
from datetime import datetime, timedelta
import random
import warnings
import os

import torch
import torch.optim as optim

from ignite.engine import Engine
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger

from lib import dqn_model, common

NAME = "01_baseline"

In [2]:
def batch_generator(buffer: ptan.experience.ExperienceReplayBuffer,
                    initial: int, batch_size: int):
    buffer.populate(initial)
    while True:
        buffer.populate(1)
        yield buffer.sample(batch_size)

In [18]:
#getting rid of missing metrics warning
warnings.simplefilter("ignore", category=UserWarning)

random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make(params.env_name)
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [19]:
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device=device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr = params.learning_rate)

In [23]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine=Engine(process_batch)
ptan_ignite.EndOfEpisodeHandler(exp_source, bound_avg_reward=15.0).attach(engine)
ptan_ignite.EpisodeFPSHandler().attach(engine)

In [24]:
@engine.on(ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)
def episode_completed(trainer: Engine):
    print("Episode %d: reward=%s, steps=%s, speed=%.3f frames/s, elapsed=%s" % (
        trainer.state.episode, trainer.state.episode_reward,
        trainer.state.episode_steps, trainer.state.metrics.get('avg_fps', 0),
        timedelta(seconds=trainer.state.metrics.get('time_passed', 0))))
    
@engine.on(ptan_ignite.EpisodeEvents.BOUND_REWARD_REACHED)
def game_solved(trainer: Engine):
    print("Game solved in %s, after %d episodes and %d iterations!" % (
        timedelta(seconds=trainer.state.metrics['time_passed']),
        trainer.state.episode, trainer.state.iteration))
    trainer.should_terminate = True
    
logdir = f"runs/{datetime.now().minute}-{params.run_name}-{NAME}"
tb = tb_logger.TensorboardLogger(log_dir=logdir)
RunningAverage(output_transform=lambda v: v['loss']).attach(engine, "avg_loss")

episode_handler = tb_logger.OutputHandler(tag='episodes', metric_names=['reward', 'steps', 'avg_reward'])
tb.attach(engine, log_handler=episode_handler, event_name=ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)

# writing to tensorboard every 100 iterations
ptan_ignite.PeriodicEvents().attach(engine)
handler = tb_logger.OutputHandler(tag="train", metric_names=['avg_loss', 'avg_fps'],
                                  output_transform=lambda a: a)
tb.attach(engine, log_handler=handler, event_name=ptan_ignite.PeriodEvents.ITERS_100_COMPLETED)

engine.run(batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-21.0, steps=778, speed=0.000 frames/s, elapsed=0:01:06.750490
Episode 2: reward=-21.0, steps=762, speed=0.000 frames/s, elapsed=0:01:06.750490
Episode 3: reward=-21.0, steps=759, speed=0.000 frames/s, elapsed=0:01:06.751493
Episode 4: reward=-20.0, steps=839, speed=0.000 frames/s, elapsed=0:01:06.751493
Episode 5: reward=-21.0, steps=755, speed=0.000 frames/s, elapsed=0:01:06.751493
Episode 6: reward=-21.0, steps=760, speed=0.000 frames/s, elapsed=0:01:06.751493
Episode 7: reward=-21.0, steps=761, speed=0.000 frames/s, elapsed=0:01:06.751493
Episode 8: reward=-21.0, steps=759, speed=0.000 frames/s, elapsed=0:01:06.752491
Episode 9: reward=-21.0, steps=759, speed=0.000 frames/s, elapsed=0:01:06.752491
Episode 10: reward=-21.0, steps=755, speed=0.000 frames/s, elapsed=0:01:06.752491
Episode 11: reward=-21.0, steps=761, speed=0.000 frames/s, elapsed=0:01:06.752491
Episode 12: reward=-21.0, steps=757, speed=0.000 frames/s, elapsed=0:01:06.753490
Episode 13: reward=-20.0,

Engine run is terminating due to exception: .


KeyboardInterrupt: 